In [35]:
def output_log_writer(s, end='\r'):
    with open(f'tmp_output_{model_name}_{date_time}.txt', 'a') as output_file:
        output_file.write(s+'\n')
        print(s,end,flush=True)

In [36]:
epoch =0
model_name = 'resnet50.a1_in1k'
from datetime import datetime
current_datetime = datetime.now()
date_time = str(current_datetime)[:-7].replace('-','').replace(':','').replace(' ','_')

In [37]:
output_log_writer(f'-------------------------------[Epoch {epoch+1}]---------------------------------')
output_log_writer(f'Training...', end='')
output_log_writer(f'Computing Train Measurement...')
output_log_writer(f'Computing Validation Measurement...')
output_log_writer(f'Model checkpoint is saved.')
output_log_writer('Final model saved. Training Finished.')

-------------------------------[Epoch 1]--------------------------------- 
Training... 
Computing Train Measurement... 
Computing Validation Measurement... 
Model checkpoint is saved. 
Final model saved. Training Finished. 


In [ ]:
    # save result
    #print(f'[Epoch {epoch+1}] Saving results... \r')
    #with open(f'model_result/log/model_loss_{date_time}.txt', 'w') as loss_file:
    #    loss_file.write(f'Epoch_{epoch+1}, {loss:.2e}\r')
    
    #with open(f'model_result/log/model_train_acc_{date_time}.txt', 'w') as acc_train_file:
    #    acc_train_file.write(f'Epoch_{epoch+1}, {accus_train[-2]:.2%}\r')
    
    #with open(f'model_result/log/model_val_acc_{date_time}.txt', 'w') as acc_val_file:
    #    acc_val_file.write(f'Epoch_{epoch+1}, {accus_val[-1]:.2%}\r')

In [38]:
str = '================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================'

len(str)

2464

In [1]:
import shutil
import os
from tqdm import tqdm
import pandas as pd
import numpy as np

In [7]:
file_name = '00003989_027.png'
train_label_map[file_name]

{'Atelectasis': 0,
 'Cardiomegaly': 1,
 'Effusion': 0,
 'Infiltration': 1,
 'Mass': 0,
 'Nodule': 0,
 'Pneumonia': 0,
 'Pneumothorax': 0,
 'Consolidation': 0,
 'Edema': 0,
 'Emphysema': 0,
 'Fibrosis': 0,
 'Pleural_Thickening': 0,
 'Hernia': 0}

In [8]:
from sklearn.metrics import roc_auc_score

In [ ]:
# 定义编码器
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, latent_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# 定义解码器
class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(latent_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# 定义自编码器
class Autoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder(input_dim, hidden_dim, latent_dim)
        self.decoder = Decoder(latent_dim, hidden_dim, input_dim)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
class Autoencoder(torch.nn.Module): 
    def __init__(self, input_dim, hidden_dim, latent_dim): 
        super().__init__()         
        self.encoder = torch.nn.Sequential( 
            torch.nn.Linear(input_dim, hidden_dim), 
            torch.nn.ReLU(), 
            torch.nn.Linear(hidden_dim, latent_dim)
        ) 
          
        self.decoder = torch.nn.Sequential( 
            torch.nn.Linear(latent_dim, hidden_dim), 
            torch.nn.ReLU(), 
            torch.nn.Linear(hidden_dim, input_dim),
            torch.nn.Sigmoid() 
        ) 
  
    def forward(self, x): 
        encoded = self.encoder(x) 
        decoded = self.decoder(encoded) 
        return decoded 

In [ ]:
# Autoencoder model
class Autoencoder(nn.Module):
    def __init__(self, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Linear(3 * 224 * 224, encoding_dim)
        self.decoder = nn.Linear(encoding_dim, 3 * 224 * 224)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.encoder(x))
        x = torch.sigmoid(self.decoder(x))
        x = x.view(x.size(0), 3, 224, 224)
        return x

In [ ]:
# Autoencoder model
class Autoencoder(nn.Module):
    def __init__(self, encoding_dim, hidden_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(3 * 224 * 224, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, encoding_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 3 * 224 * 224),
            nn.Sigmoid()
        )
 
 
    def forward(self, x):
        batchsz = x.size(0)
        # flatten
        x = x.view(batchsz, -1)
        # encoder
        x = self.encoder(x)
        # decoder
        x = self.decoder(x)
        # reshape
        x = x.view(batchsz, 3, 224, 224)
        
        return x  

In [ ]:
# class CAE(nn.Module):
#     def __init__(self, encoding_dim, hidden_dim):
#         super(CAE, self).__init__()
#         self.encoder = nn.Sequential(
#             nn.Conv2d(3, hidden_dim, 3, stride=1, padding=1),  # b, 16, 10, 10
#             nn.ReLU(True),
#             nn.MaxPool2d(2, stride=2),  # b, 16, 5, 5
#             nn.Conv2d(hidden_dim, encoding_dim, 3, stride=1, padding=1),  # b, 8, 3, 3
#             nn.ReLU(True),
#             nn.MaxPool2d(2, stride=2)  # b, 8, 2, 2
#         )
#         self.decoder = nn.Sequential(
#             nn.ConvTranspose2d(encoding_dim, hidden_dim, 3, stride=2, padding=1, output_padding=1),  # b, 16, 5, 5
#             nn.ReLU(True),
#             nn.ConvTranspose2d(hidden_dim, 224, 3, stride=2, padding=1, output_padding=1),  # b, 8, 15, 15
#             nn.ReLU(True),
#             nn.ConvTranspose2d(224, 3, 3, stride=1, padding=1),  # b, 1, 28, 28
#             nn.Tanh()
#         )
 
#     def forward(self, x):
#         x = self.encoder(x)
#         x = self.decoder(x)


In [ ]:
# # check the flow of the structure
# net = nn.Sequential(
#     nn.Conv2d(3, 128, 3, stride=1, padding=1), nn.ReLU(True), nn.MaxPool2d(2, stride=2),
#     nn.Conv2d(128, 32, 3, stride=1, padding=1), nn.ReLU(True), nn.MaxPool2d(2, stride=2),
#     nn.ConvTranspose2d(32, 128, 3, stride=2, padding=1, output_padding=1), nn.ReLU(True),
#     nn.ConvTranspose2d(128, 224, 3, stride=2, padding=1, output_padding=1), nn.ReLU(True),
#     nn.ConvTranspose2d(224, 3, 3, stride=1, padding=1), nn.Tanh())

# X = torch.rand(size=(1,3,224,224), dtype=torch.float32)
# for layer in net:
#     print(layer.__class__.__name__,'output shape: \t',X.shape)
#     X = layer(X)